In [1]:
# pip install googletrans
# pip install openai
# pip install nltk
# pip install gtts

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import os
import openai
import spacy
from spacy.matcher import Matcher
import tensorflow as tf
from keras.optimizers import SGD
from keras.preprocessing.sequence import pad_sequences

nlp = spacy.load("en_core_web_sm")
matcher=Matcher(nlp.vocab)

# from googletrans import Translator
# import time
# from gtts import gTTS

import numpy as np
from keras.layers import Dense, Activation, Dropout
import random
from keras.models import load_model, Sequential

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json', encoding='utf-8').read()
intents = json.loads(data_file)


In [3]:
openai.api_key = <your_api_key>

def api(message):
    # Get the message from the POST request
    message = message
    # Send the message to OpenAI's API and receive the response
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": message}
        ]
    )
    if completion.choices[0].message != None:
        return completion.choices[0].message.content

    else:
        return 'Failed to Generate response!'

In [4]:
nltk.download('punkt')

for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\20520\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
nltk.download('wordnet')

# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
max=len(words)
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\20520\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


204 documents
45 classes ['App di động', 'cancel', 'chào hỏi', 'chậm trễ', 'confirm', 'cá nhân', 'cảm ơn', 'delay', 'delivery', 'giao hàng', 'goodbye', 'greeting', 'hoàn trả', 'hủy', 'items', 'lí do', 'mobile app', 'more', 'mặt hàng', 'order status', 'payments', 'personal', 'phản hồi khách hàng', 'profile info', 'reasons', 'refund', 'refund status', 'return', 'rút tiền hoàn trả', 'sử dụng voucher', 'thanh toán', 'thanks', 'theo dõi', 'thêm', 'thông tin khách hàng', 'track', 'trạng thái hoàn trả', 'trạng thái đơn hàng', 'trả lại', 'tạm biệt', 'use voucher', 'user response', 'withdraw refunds', 'xác nhận', 'đặt hàng']
356 unique lemmatized words ["'m", "'s", ',', '.', '012345678', '0702245557', '0946805379', '1', '1p', '30', '6h15', '7h', '8h,0981148781', 'a', 'about', 'accept', 'account', 'address', 'agent', 'alo', 'am', 'an', 'any', 'anyone', 'app', 'are', 'available', 'b', 'b4', 'bao', 'be', 'become', 'biến', 'biết', 'biệt', 'but', 'buy', 'bye', 'bán', 'bánh', 'bạn', 'bật', 'bằng', 'b

In [6]:
# Define the maximum length for your bag-of-words lists
max_length = max

training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Pad the bag-of-words list to the specified max_length
    padded_bag = pad_sequences([bag], maxlen=max_length, padding='post')[0]

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([padded_bag, output_row])

# Convert training data to NumPy arrays
random.shuffle(training)
train_x_np, train_y_np = zip(*training)
train_x = np.array(train_x_np)
train_y = np.array(train_y_np)

print("Training data created")


Training data created


In [7]:
np.set_printoptions(threshold=np.inf)

In [8]:
#define model

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [9]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = tf.keras.optimizers.legacy.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

c:\Users\20520\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [10]:
# fitting and saving the mode
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=32, verbose=1)
model.save('model.h5', hist)

Epoch 1/200
7/7 [==============================] - 0s 2ms/step - loss: 3.8052 - accuracy: 0.0049
Epoch 2/200
7/7 [==============================] - 0s 2ms/step - loss: 3.7883 - accuracy: 0.0294
Epoch 3/200
7/7 [==============================] - 0s 2ms/step - loss: 3.7588 - accuracy: 0.0441
Epoch 4/200
7/7 [==============================] - 0s 2ms/step - loss: 3.7176 - accuracy: 0.0539
Epoch 5/200
7/7 [==============================] - 0s 2ms/step - loss: 3.6814 - accuracy: 0.1078
Epoch 6/200
7/7 [==============================] - 0s 2ms/step - loss: 3.6546 - accuracy: 0.1029
Epoch 7/200
7/7 [==============================] - 0s 2ms/step - loss: 3.6022 - accuracy: 0.1176
Epoch 8/200
7/7 [==============================] - 0s 2ms/step - loss: 3.5803 - accuracy: 0.1225
Epoch 9/200
7/7 [==============================] - 0s 2ms/step - loss: 3.5287 - accuracy: 0.1275
Epoch 10/200
7/7 [==============================] - 0s 2ms/step - loss: 3.4817 - accuracy: 0.1029
Epoch 11/200
7/7 [===========

c:\Users\20520\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
model = load_model('model.h5')
intents = json.loads(open('intents.json', encoding='utf-8').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [12]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that

In [13]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("Index of words: ", i)
                    print("Words: ", w)
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [14]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=True)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.8
    print("Result: ", res.shape)
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    
    print("Results: ", results)
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    if results:
        return return_list, results[0][0]
    else:
        return return_list, None


In [15]:
def get_information_ordering(doc):
    patterns = [
        [{"POS": "NUM"}, {"OP": "?"}, {"POS": "NOUN"}], # Pattern for quantity and product
        [{"POS": "NUM"}, {"OP": "?"}, {"POS": "PROPN"}],
        [{"LOWER": "size"}, {"POS": "NOUN"}],  # Pattern for size
        [{"LOWER": "size"}, {"POS": "PROPN"}],
        [{"LOWER": "sdt"}, {"POS": "NUM"}],
        [{"LOWER": "thoại"}, {"POS": "NUM"}],
        [{"LOWER": "chỉ"}, {"POS": "PROPN"}],
        [{"LOWER": "dc"}, {"POS": "PROPN"}],
        [{"LOWER": "chỉ"}, {"POS": "NOUN"}],
        [{"LOWER": "dc"}, {"POS": "NOUN"}],
        [{"LOWER": "giao"}, {"POS": "NOUN"}],
        [{"LOWER": "giao"}, {"POS": "PROPN"}],
        [{"POS": "NOUN"}, {"POS": "NUM"}],
        [{"POS": "PUNCT"}, {"POS": "NUM"}]
    ]

    matcher = Matcher(nlp.vocab)

    for pattern in patterns:
        matcher.add("pattern", [pattern])

    matches = matcher(doc)

    quantity = ""
    product = []
    size = ""
    phone=""
    address=[]
    found_address = False

    for match_id, start, end in matches:
        matched_span = doc[start:end]
        if matched_span[0].pos_ == "NUM":
            if quantity == "":
                quantity = matched_span[0].text
                if matched_span[1].text in ["ly", "cái", "chiếc", "phần"]:
                    for i in range(start+2, len(doc)):
                        if doc[i].pos_!='NOUN' and doc[i].pos_!='PROPN':
                            break
                        if doc[i].text=='size':
                            break
                        else:
                            product.append(doc[i].text)
                    # product=doc[end].text
                else:
                    for i in range(start+1, len(doc)):
                        if doc[i].text=='size':
                            break
                        else:
                            product.append(doc[i].text)

        elif matched_span[0].lower_ == "size" and len(matched_span) > 1:
            size = matched_span[1].text
        elif matched_span[0].pos_ == "NUM" and matched_span[1].pos_ != "NOUN" or matched_span[1].pos_ != "PROPN":
            phone = matched_span[1].text
        elif matched_span[0].lower_ == "sdt" and matched_span[1].pos_ == "NUM":
            phone = matched_span[1].text
        elif matched_span[0].lower_ == "thoại" and matched_span[1].pos_ == "NUM":
            phone = matched_span[1].text
        elif matched_span[0].lower_ == "chỉ" and matched_span[1].pos_ == "PROPN" or matched_span[1].pos_ == "NOUN":
            if found_address:
                continue
            found_address = False
            for i in range(start+1, len(doc)):
                if doc[i].pos_ != 'NOUN' and doc[i].pos_ != 'PROPN':
                    found_address = True
                    break
                else:
                    address.append(doc[i].text)
        elif matched_span[0].lower_ == "dc" and matched_span[1].pos_ == "PROPN" or matched_span[1].pos_ == "NOUN":
            if found_address:
                continue
            found_address = False
            for i in range(start+1, len(doc)):
                if doc[i].pos_ != 'NOUN' and doc[i].pos_ != 'PROPN':
                    found_address = True
                    break
                else:
                    address.append(doc[i].text)
        elif matched_span[0].lower_ == "giao" and matched_span[1].pos_ == "PROPN" or matched_span[1].pos_ == "NOUN":
            if found_address:
                continue
            found_address = False
            for i in range(start+1, len(doc)):
                if doc[i].pos_ != 'NOUN' and doc[i].pos_ != 'PROPN':
                    found_address = True
                    break
                else:
                    address.append(doc[i].text)
            

    return product, quantity, size, phone, address

In [16]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [17]:
def chatbot_response(text):
    product = []
    quantity = ""
    size = ""
    phone=""
    add=""
    
    ints, index = predict_class(text, model)
    print(len(ints))
    if len(ints) == 0:
        res = api(text)
    else:
        res = getResponse(ints, intents)
        if index==44:
            # Phân tích câu và trích xuất thông tin
            doc = nlp(text)
            for token in doc:
                print(token.pos_)
            product, quantity, size, phone, add = get_information_ordering(doc)

    return res, [product, quantity, size, phone, add]

In [18]:
# def text_to_speech(text):
#     detector=Translator()
#     dec_lan=detector.detect(text)
#     if dec_lan.lang=='vi':
#         speaker = gTTS(text=text, lang="vi", slow=False)
#     else:
#         speaker = gTTS(text=text, lang="en", slow=False)

#     speaker.save("res.mp3")
#     statbuf = os.stat("res.mp3")
#     mbytes = statbuf.st_size / 1024
#     duration = mbytes / 200
#     os.system('start res.mp3')  #if you are using mac->afplay or else for windows->start
#     # os.system("close res.mp3")
#     time.sleep(int(50*duration))
#     os.remove("res.mp3")

In [19]:
import tkinter
from tkinter import *
import csv
import openpyxl
from pathlib import Path

BG_GRAY = "#ABB2B9"
BG_COLOR = "#BCD9FF"
TEXT_COLOR = "#000000"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 15 bold"

def send(event):
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You:" +' ', ("small", "right", "greycolour"))
        ChatLog.window_create(END, window=Label(ChatLog, fg="#000000", text=msg, 
        wraplength=500, font=("Arial", 10), bg="lightblue", bd=4, justify="left"))
        ChatLog.insert(END,'\n\n', "left")
        ChatLog.config(foreground="#0000CC", font=("Helvetica", 9))
        ChatLog.yview(END)

        res, data = chatbot_response(msg)
        ChatLog.insert(END, "Bot:"+' ', ("small", "greycolour", "left"))
        ChatLog.window_create(END, window=Label(ChatLog, fg="#000000", text=res, 
        wraplength=500, font=("Arial", 10), bg="#DDDDDD", bd=4, justify="left"))
        ChatLog.insert(END, '\n\n', "right")
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

        print(data)
        if data and any(value != '' for value in data if value != []):
            if type(data[0]) == list:
                data[0] = " ".join(data[0])
            if type(data[4]) == list:
                data[4] = " ".join(data[4])
                print(type(data[4]))
            filename = "product_info.xlsx"
            fields = ["Product", "Quantity", "Size", "Phone", "Address"]

            file_path = Path(filename)

            if file_path.exists():
                wb = openpyxl.load_workbook(filename)
                ws = wb.active
            else:
                wb = openpyxl.Workbook()
                ws = wb.active
                ws.append(fields)

            ws.append(data)

            wb.save(file_path)

base = Tk()
base.title("E-Commerce Chatbot")
base.resizable(width=True, height=True)
base.configure(width=800, height=800, bg=BG_COLOR)


#Create Chat window
ChatLog = Text(base, bd=10, bg="#fff7f4", fg=TEXT_COLOR, font=FONT_BOLD)
ChatLog.config(state=DISABLED)

head_label = Label(base, bg=BG_COLOR, fg=TEXT_COLOR, 
                    text="Welcome to The Buziers's Chatbot", 
                    font=FONT_BOLD, pady=10)
head_label.place(relwidth=1)

line = Label(base, width=450, bg=BG_COLOR)
 
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
ChatLog.focus()

#Create the box to enter message
EntryBox = Text(base, bg="white",width="29", height="5", font="Arial", background="#dddddd")
EntryBox.focus()
EntryBox.bind("<Return>", send)

#Create Button to send message
SendButton = Button(base, font=("Verdana", 12,'bold'), text="Send", width="12", height=15,
                    bd=0, bg="#ed9061", activebackground="#3c9d9b",fg='#ffffff',
                    command=lambda: send)

scrollbar.place(x=775,y=6, height=800)
line.place(x=0,y=35, height=1, width=770)
ChatLog.place(x=5,y=40, height=700, width=770)
EntryBox.place(x=0, y=740, height=60, width=600)
SendButton.place(x=600, y=740, height=60, width=175)

In [20]:
base.mainloop()

Index of words:  328
Words:  xin
found in bag: xin
Index of words:  57
Words:  chào
found in bag: chào
1/1 [==============================] - 0s 64ms/step
Result:  (45,)
Results:  [[2, 0.9561474]]
1
[[], '', '', '', '']
Index of words:  117
Words:  hello
found in bag: hello
1/1 [==============================] - 0s 16ms/step
Result:  (45,)
Results:  [[11, 0.9574409]]
1
[[], '', '', '', '']
Index of words:  297
Words:  tôi
found in bag: tôi
Index of words:  338
Words:  đang
found in bag: đang
Index of words:  74
Words:  cảm
found in bag: cảm
Index of words:  273
Words:  thấy
found in bag: thấy
Index of words:  353
Words:  ổn
found in bag: ổn
1/1 [==============================] - 0s 14ms/step
Result:  (45,)
Results:  [[22, 0.9836887]]
1
[[], '', '', '', '']
Index of words:  256
Words:  thank
found in bag: thank
Index of words:  333
Words:  you
found in bag: you
1/1 [==============================] - 0s 15ms/step
Result:  (45,)
Results:  [[31, 0.9773399]]
1
[[], '', '', '', '']
Index of 